In [1]:
Packages <- c("dplyr","nleqslv", "broom","cubature", "geosphere", "data.table",  "ggplot2", "bbmle", "dplyr","tidyr", "stringr", "tidyverse", "lubridate", "RColorBrewer")

invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/oceanography')
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0
source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_bbmle.R")
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum0.5.R") #integrate_kernel_sum1
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum1.R") #integrate_kernel_sum0.5
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve.R") #median
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve90.R") #dist 90% retained

#clownfish metadata
load("~/parentage/r_data/total_sampling_across_years.RData")
load("~/parentage/r_data/sampled_area_each_year.RData")
#https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Script_outputs/cumulative_prop_hab_sampled_by_site.RData)
load("~/parentage/r_data/cumulative_prop_hab_sampled_by_site.RData")
#download.file(url = "https://github.com/pinskylab/genomics/blob/master/data/fish-obs.RData?raw=true", destfile = "~/parentage/r_data/fish-obs.RData")
fish_obs <- readRDS("~/parentage/r_data/fish-obs.RData") 
load("~/parentage/r_data/site_dist_info.RData")
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Data_from_database/anem_db.RData?raw=true", destfile = "~/parentage/r_data/anem_db.RData")
load("~/parentage/r_data/anem_db.RData")
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Data_from_database/dives_db.RData?raw=true", destfile = "~/parentage/r_data/dives_db.RData")
load("~/parentage/r_data/dives_db.RData")
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Data_from_database/fish_db.RData?raw=true", destfile = "~/parentage/r_data/dives_db.RData")
load("~/parentage/r_data/fish_db.RData")
load("~/parentage/r_data/gps_db.RData")

#stop dplyr's courtesy warnings about grouping variables and summarise()
options(dplyr.summarise.inform=F) 

__Narrow down data to the year of interests__

In [2]:
UnsampledSites <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6", "Pangasugan", "CAI", "Other") 


In [3]:

#read in the df where each particle is an observation, filter out sand flats as sources
SandFlats <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6")
Simulation2012 <- fread(file="~/oceanography/script_output/SimulationSummaryTables/SimConnLongForm2012YearSampled.csv", showProgress = T) %>%
    filter(source %!in% SandFlats)
Simulation2013 <- fread(file="~/oceanography/script_output/SimulationSummaryTables/SimConnLongForm2013YearSampled.csv", showProgress = T) %>%
    filter(source %!in% SandFlats)
Simulation2014 <- fread(file="~/oceanography/script_output/SimulationSummaryTables/SimConnLongForm2014YearSampled.csv", showProgress = T) %>%
    filter(source %!in% SandFlats)

#this is about 3 GB so fread is fastest, but consider these options for dealing with a large csv
#https://inbo.github.io/tutorials/tutorials/r_large_data_files_handling/

#head(Simulation2012 %>% group_by(date, source, destination) %>% summarise(Check=n()))
#CAI should be 201718, it is, all good

In [4]:
#read in the centroids adjusted for the simulation, so the Magbangons combined 
#centroids <- read.csv(file="~/oceanography/script_output/SurveyData/SimulationCentroids.csv", header=T, stringsAsFactors = F)
#read in the distance matrix adjusted for the simulation
#distances <- read.csv(file="~/oceanography/script_output/SurveyData/SimulationAllDistances.csv", header=T, stringsAsFactors = F)


#proportion sampled
#add how well we sampled sites- proportion of habitat sampled
load("~/parentage/r_data/cumulative_prop_hab_sampled_by_site.RData")
PropSamp <- cumulative_prop_hab_sampled_by_site %>%
    mutate(total_possible_sample_anems = ifelse(site=="Caridad Proper", 4, total_possible_sample_anems) ) %>%
    mutate(total_prop_hab_sampled_anems_tidied= ifelse(site=="Caridad Proper" & total_anems_sampled==4, 1, total_prop_hab_sampled_anems_tidied) ) %>%
    mutate(total_possible_sample_anems = ifelse(site=="Sitio Lonas", total_anems_sampled, total_possible_sample_anems) ) %>%
    mutate(total_prop_hab_sampled_anems_tidied= ifelse(site=="Sitio Lonas", 1, total_prop_hab_sampled_anems_tidied) )%>%
    mutate(total_prop_hab_sampled_anems_tidied= ifelse(is.nan(total_prop_hab_sampled_anems_tidied), 0, total_prop_hab_sampled_anems_tidied) ) %>%
    dplyr::select(site, time_frame, end_year, total_prop_hab_sampled_anems_tidied) 

PropSamp$site <- gsub(". ", ".", PropSamp$site, fixed=TRUE) #fix spaces in Magbangon names

S.Mag <- PropSamp %>%#make a table for all of the S. Mabangon years/prop sampled. Then join to the DF, make a column adding S.Magbangon values to all rows of prop sampled, but only sub that value in for PropSamp in N.Magbangon rows, then rename N.Magbangon as Magbangon
    filter(site=="S.Magbangon") %>%
    dplyr::select(site, end_year, total_prop_hab_sampled_anems_tidied) %>%
    rename(S.MagVal="total_prop_hab_sampled_anems_tidied", extra="site")

PropSamp <- left_join(PropSamp, S.Mag, by=c("end_year")) %>%
    mutate(S.MagSum=total_prop_hab_sampled_anems_tidied+S.MagVal) %>% #create col adding the S.Mag values to prop hab - keep in mind values could be greater than 1- if so change them to 1
    mutate(total_prop_hab_sampled_anems_tidied= ifelse(site=="N.Magbangon", S.MagSum, total_prop_hab_sampled_anems_tidied)) %>%#sub this value in for only N.Mag
    mutate(site=ifelse(site=="N.Magbangon", "Magbangon", site)) %>%#change N.Mag name to generic Mag
    filter(site !="S.Magbangon") %>%#eliminate the S.Mag rows, they are now repeats
    dplyr::select(-S.MagVal,-extra, -S.MagSum, -time_frame) %>%
    rename(PropSamp="total_prop_hab_sampled_anems_tidied")


#check if there are values>1, should be none
sum(which(PropSamp$total_prop_hab_sampled_anems_tidied >1)) #zero, that's good

[1] 0

In [5]:
#load data from the genetic sampling in each year
#read in the kernel fitting summary
kernels <- read.csv(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/kernel_fitting_summary.csv", header=T, stringsAsFactors = F)

##read in the sites that we sampled each year
N_gen_par <- read.table(file="~/parentage/colony2/20190523_1340loci/input/all_parents_corrected.txt", header = TRUE, stringsAsFactors = F) %>%#not sure that I need the parents here
    mutate(fish_indiv=as.character(fish_indiv))
N_gen_offs <- read.table(file="~/parentage/colony2/20190523_1340loci/input/all_offspring_corrected.txt", header=T, stringsAsFactors = F) %>%
    mutate(fish_indiv=as.character(fish_indiv))



#gather the summary of total offspring sampled
#from Allison, putting all the meta data together (Constants_database_common_functions.R)
##### Match up other relevant info (site, date, fish_indiv, etc.) to fish in the clownfish table
# Pull out year and month into a separate column in dives_db
dives_db_processed <- dives_db %>%
  mutate(year = as.integer(substring(date,1,4))) %>%
  mutate(month = as.integer(substring(date,6,7))) %>%
  mutate(dive_date = date(date))

# Pull all APCL caught or otherwise in the clownfish table
allfish_fish <- fish_db %>%
  select(fish_table_id, anem_table_id, fish_spp, sample_id, anem_table_id, recap, tag_id, color, sex, size, fish_obs_time, fish_notes) %>%
  filter(fish_spp == 'APCL') %>%
  mutate(size = as.numeric(size))  # make the size numeric (rather than chr) so can do means and such

# and their corresponding anemones
allfish_anems <- anem_db %>%
  select(anem_table_id, dive_table_id, anem_obs, anem_id, old_anem_id, anem_notes) %>%
  filter(anem_table_id %in% allfish_fish$anem_table_id)

# and the corresponding dive info
allfish_dives <- dives_db_processed %>%
  select(dive_table_id, dive_type, date, year, month, site, gps, dive_notes) %>%
  filter(dive_table_id %in% allfish_anems$dive_table_id) 

# join together
allfish_caught <- left_join(allfish_fish, allfish_anems, by="anem_table_id")
allfish_caught <- left_join(allfish_caught, allfish_dives, by="dive_table_id")

# add in the gen_ids and fish_indiv (now in a separate gen_id table) - gen_id only comes in the time the fish was sequenced, not at all captures
allfish_caught <- left_join(allfish_caught, (fish_obs %>% select(fish_table_id, gen_id, fish_indiv)), by = "fish_table_id") %>%
    select(fish_indiv, sample_id, site) %>%
    mutate()

N_gen_offs_annual  <- left_join(N_gen_offs, allfish_caught, by=c("fish_indiv", "sample_id")) %>% 
    group_by(year, site) %>%
    summarise(n_offs_gen=n()) %>%
    ungroup()

N_gen_offs_annual$site <- gsub(". ", ".", N_gen_offs_annual$site, fixed=TRUE)

##for all years
NGenOffsAll <- N_gen_offs_annual %>% 
    group_by(site) %>% 
    summarise(n_offs_gen=sum(n_offs_gen, na.rm=T))

sum(NGenOffsAll$n_offs_gen)

[1] 791

In [6]:
#combine N/S Magbangon in the genetic sampling data

AnnualRecsSamp <- bind_rows(N_gen_offs_annual %>%
                        filter(site !="N.Magbangon" & site!="S.Magbangon"),
                    N_gen_offs_annual %>%
                        mutate(Magbangon=ifelse(site=="N.Magbangon" | site=="S.Magbangon", "Magbangon", "no")) %>%
                        filter(Magbangon=="Magbangon") %>%
                        group_by(year, Magbangon) %>%
                        summarise(sum_offs=sum(n_offs_gen)) %>%
                        rename(site="Magbangon", n_offs_gen="sum_offs")) %>%
                        mutate(year=as.character(year)) %>%
                        ungroup()
sum(AnnualRecsSamp$n_offs_gen) #should be 791

#add in ALL sites that we sampled, even though we didn't find recruits there
AnnualRecsSamp <- left_join(PropSamp %>%  #add in all sampled sites as possible offs sites
                            filter(PropSamp > 0),
                          AnnualRecsSamp, 
                              by=c("site", end_year="year")) %>%
                        rename(year="end_year")  %>%
                        mutate(n_offs_gen=ifelse(is.na(n_offs_gen), 0, n_offs_gen)) %>%#if we didn't sample a recruit at a site that we still visited, put a 0 in place of the NA
                        filter(year %in% c("2012", "2013", "2014")) %>%  #* restrict to what we can pick up combining all years of simulation data
                        group_by(site) %>% #*
                        mutate(n_offs_gen_all_years=sum(n_offs_gen)) #*


[1] 791

__Narrow down data to the year and adjustments of interests__

In [19]:
#what if we account for not sampling new recruits?
Simulation2012NoRecentRec <- Simulation2012 %>%
        filter(SimMonth %!in% c(4,5))#account for the fact that we didn't sample really new recruits
    
nrow(Simulation2012NoRecentRec)<nrow(Simulation2012)#should be TRUE

[1] TRUE

__Get all the components of the kernel fitting process together__

In [20]:
#assemble all of the components for annual kernel fitting, search and replace for the years
NGenOffs2012 <- AnnualRecsSamp %>%
    filter(year=="2012")

nrow(NGenOffs2012) #should be 9 sites sampled for 2012

#***
PropSamp2012 <- PropSamp %>%  #add in all sampled sites as possible parent sites
                filter(end_year =="2012" & PropSamp > 0) %>%
                #rename(source="site") %>%
                select(-end_year, -PropSamp)

PropSamp2012 <- PropSamp2012 %>%
                rename(source="site")

#PropSamp2012$destination <- PropSamp2012$source #make another column for destination
#***
centroids <- read.csv("~/oceanography/empirical_data/site_centroids_simulation_kernels.csv", header=TRUE, stringsAsFactors = F)

Area <- read.csv("~/oceanography/empirical_data/site_area_header_nonsurveyed_simulation_kernels.csv", header=TRUE, stringsAsFactors = F) %>%
    arrange(site) %>%
    filter(site %!in% c("near_north_full1", "near_north_full2", "near_north_full3", "near_south_full1", "near_south_full2", "near_south_full3")) %>%
    mutate(kmsq=msq*10^-6) %>%
    select(kmsq)
Reef_sizes <- as.matrix(Area)

#give every site in the distance matrix of the simulation (even if we didn't sample there) an index number
AllSites <- centroids %>%
    select(site) %>%
    arrange(site)
nrow(AllSites) #should be 34x1
AllSites$index <- seq(from=1, to=34, by=1)

SampledSites2012 <- inner_join(PropSamp2012 %>% select(source), AllSites, by=c(source="site")) 

#check for correct number of rows
nrow(SampledSites2012)==nrow(PropSamp2012) #should be true
SampledSites2012Index <- t(as.matrix(SampledSites2012$index))
ncol(SampledSites2012Index)

Sampled_reefs2012 <- SampledSites2012Index

#proportion sampled matrix for kernel fitting

#for annual
Adult_sample_proportions2012 <- PropSamp %>%  #add in all sampled sites as possible parent sites
                filter(end_year =="2012" & PropSamp > 0) 

Adult_sample_proportions2012 <- as.matrix(Adult_sample_proportions2012$PropSamp)
nrow(Adult_sample_proportions2012) #should be 9 for 2012

#distance matrix using the centroids with combined Magbangon
### List of source locations
sites_source <- centroids

### List of destination locations
sites_dest <- centroids

dist_mat_m <- distm(sites_source[,c('lon','lat')], sites_source[,c('lon','lat')], fun=distVincentyEllipsoid)
Distances <- dist_mat_m*10^-3

Centroids <- centroids %>%
    select(-site)



[1] 9

[1] 34

[1] TRUE

[1] 9

[1] 9

In [ ]:

#awesome! everything works! let's rip with a 10 iteration for loop
col <- c("year", "k", "theta", "mdd", "med", "dist90", "iteration")
SimulatedKernels2012 <- as.data.frame(matrix(nrow=0, ncol=7), stringsAsFactors = FALSE)
colnames(SimulatedKernels2012) <- col

#don't print warnings for this loop, they are only for setting row names in a tibble which is depracated. But it works. 
options(warn=-1)

pb <- txtProgressBar(min = 0, max = 1000, style = 3)

StartTime <- Sys.time()

for(n in 1:1000){
        
    #SAMPLE THE SIMULATION DATA, then fit a kernel
    
    #make an empty dataframe with the same columns as original, this will be sampled particles
    SimulatedSampling2012 = Simulation2012[FALSE,]
    
    for(i in 1:nrow(NGenOffs2012)){ 
        
        destination_eval <- as.character(NGenOffs2012$site[i]) #pick out a destination site
        
        SimSampDestination <- Simulation2012NoRecentRec %>% 
                                filter(destination==destination_eval) %>%
                                sample_n(as.numeric(NGenOffs2012$n_offs_gen[i]),  replace=F) 
        
        SimulatedSampling2012 <- bind_rows(SimulatedSampling2012, SimSampDestination) #build into a sampled particle df
    }
    
    #assign a numeric ID for each row (which is a sampled particle)
    SimulatedSampling2012 <- SimulatedSampling2012 %>%
        mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))
    
    #nrow(SimulatedSampling2012)==sum(NGenOffs2012$n_offs_gen) #should be TRUE
    
    #now randomly assign parentage match status to some of these rows
    NumPar <- as.numeric(kernels %>%
        filter(Year=="2012") %>%
        select(NumParentageMatches))
    
    SimulatedSampling2012Par <- SimulatedSampling2012 %>%
        ungroup() %>%
        filter(source %in% NGenOffs2012$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
        sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
        mutate(Parentage=1) #%>% #assign these rows a positive match value
        #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
        #select(date, source, destination, Parentage) #drop columns that will result in repeats 
    
    #find the rows that haven't been assigned parentage
    SimulatedSampling2012_2 <- anti_join((SimulatedSampling2012 %>% ungroup()), SimulatedSampling2012Par, by=c("ParticleSampID")) 
    
    SimulatedSampling2012_2$Parentage <- 0 #add in the column for parentage
    
    #combine parentage and non-parentage dfs
    SimulatedSampling2012Par3 <- bind_rows(SimulatedSampling2012Par, SimulatedSampling2012_2) %>%
        mutate(YearSampled=as.character(YearSampled))
    
    
    #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSamp2012 <- PropSamp %>%  #add in all sampled sites as possible parent sites
                            filter(end_year =="2012" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSamp2012$destination <- PropSamp2012$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSampling2012Par3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in all surveyed sites
                TotalSimSamp2 <- left_join(PropSamp2012, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSampling2012Par3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum all the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummary2012 <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum all the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finally, join all of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for all sites
                PropSamp2012 <- PropSamp2012 %>%
                                rename(source="site")
                
                PropSamp2012$destination <- PropSamp2012$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSamp2012, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% NGenOffs2012$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummary2012 %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFull2012 <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFull2012) <- NULL

        #The full remade parentage matrix
        Assignments2012 <- SimDispMatFull2012
    
    #NOW FIT THE KERNEL


x <- list(Distances=Distances, Assignments=Assignments2012, Sampled_reefs=Sampled_reefs2012, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportions2012) #put inputs into a list because that's the bbmle format

Sim2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))

BestK2012 <- as.numeric(coef(Sim2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Sim2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
    
    #store the info in this df
    SimulatedKernels2012_beta <- as.data.frame(matrix(nrow=1, ncol=7), stringsAsFactors = FALSE)
    colnames(SimulatedKernels2012_beta) <- col
    
    SimulatedKernels2012_beta$year <- 2012
    SimulatedKernels2012_beta$k <- BestK2012
    SimulatedKernels2012_beta$theta <- BestTheta2012
    SimulatedKernels2012_beta$mdd <- MDD2012
    SimulatedKernels2012_beta$med <- Med2012
    SimulatedKernels2012_beta$dist90 <- Dist90_2012
    SimulatedKernels2012_beta$iteration <- n
    
    
#join results into larger df
SimulatedKernels2012 <- bind_rows(SimulatedKernels2012, SimulatedKernels2012_beta)
    
setTxtProgressBar(pb, n)


}
close(pb)
EndTime <- Sys.time()
EndTime-StartTime
options(warn=0) #turn warnings back on


write.csv(SimulatedKernels2012, file="~/oceanography/script_output/KernelFits/Adjust_NoRecentRecruits/1000SimulatedKernels2012NoRecentRec.csv", row.names=F)
SimulatedKernels2013 <- as.data.frame(matrix(nrow=0, ncol=7), stringsAsFactors = FALSE)
colnames(SimulatedKernels2013) <- col

#don't print warnings for this loop, they are only for setting row names in a tibble which is depracated. But it works. 
options(warn=-1)

pb <- txtProgressBar(min = 0, max = 1000, style = 3)

StartTime <- Sys.time()

for(n in 1:1000){
        
    #SAMPLE THE SIMULATION DATA, then fit a kernel
    
    #make an empty dataframe with the same columns as original, this will be sampled particles
    SimulatedSampling2013 = Simulation2013[FALSE,]
    
    for(i in 1:nrow(NGenOffs2013)){ 
        
        destination_eval <- as.character(NGenOffs2013$site[i]) #pick out a destination site
        
        SimSampDestination <- Simulation2013NoRecentRec %>% 
                                filter(destination==destination_eval) %>%
                                sample_n(as.numeric(NGenOffs2013$n_offs_gen[i]),  replace=F)
        
        SimulatedSampling2013 <- bind_rows(SimulatedSampling2013, SimSampDestination) #build into a sampled particle df
    }
    
    #assign a numeric ID for each row (which is a sampled particle)
    SimulatedSampling2013 <- SimulatedSampling2013 %>%
        mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))
    
    #nrow(SimulatedSampling2013)==sum(NGenOffs2013$n_offs_gen) #should be TRUE
    
    #now randomly assign parentage match status to some of these rows
    NumPar <- as.numeric(kernels %>%
        filter(Year=="2013") %>%
        select(NumParentageMatches))
    
    SimulatedSampling2013Par <- SimulatedSampling2013 %>%
        ungroup() %>%
        filter(source %in% NGenOffs2013$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
        sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
        mutate(Parentage=1) #%>% #assign these rows a positive match value
        #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
        #select(date, source, destination, Parentage) #drop columns that will result in repeats 
    
    #find the rows that haven't been assigned parentage
    SimulatedSampling2013_2 <- anti_join((SimulatedSampling2013 %>% ungroup()), SimulatedSampling2013Par, by=c("ParticleSampID")) 
    
    SimulatedSampling2013_2$Parentage <- 0 #add in the column for parentage
    
    #combine parentage and non-parentage dfs
    SimulatedSampling2013Par3 <- bind_rows(SimulatedSampling2013Par, SimulatedSampling2013_2) %>%
        mutate(YearSampled=as.character(YearSampled))
    
    
    #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSamp2013 <- PropSamp %>%  #add in all sampled sites as possible parent sites
                            filter(end_year =="2013" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSamp2013$destination <- PropSamp2013$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSampling2013Par3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in all surveyed sites
                TotalSimSamp2 <- left_join(PropSamp2013, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSampling2013Par3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum all the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummary2013 <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum all the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finally, join all of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for all sites
                PropSamp2013 <- PropSamp2013 %>%
                                rename(source="site")
                
                PropSamp2013$destination <- PropSamp2013$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSamp2013, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% NGenOffs2013$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummary2013 %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFull2013 <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFull2013) <- NULL

        #The full remade parentage matrix
        Assignments2013 <- SimDispMatFull2013
    
    #NOW FIT THE KERNEL


x <- list(Distances=Distances, Assignments=Assignments2013, Sampled_reefs=Sampled_reefs2013, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportions2013) #put inputs into a list because that's the bbmle format

Sim2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))

BestK2013 <- as.numeric(coef(Sim2013Fit)[1])
BestTheta2013 <- as.numeric(coef(Sim2013Fit)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
    
    #store the info in this df
    SimulatedKernels2013_beta <- as.data.frame(matrix(nrow=1, ncol=7), stringsAsFactors = FALSE)
    colnames(SimulatedKernels2013_beta) <- col
    
    SimulatedKernels2013_beta$year <- 2013
    SimulatedKernels2013_beta$k <- BestK2013
    SimulatedKernels2013_beta$theta <- BestTheta2013
    SimulatedKernels2013_beta$mdd <- MDD2013
    SimulatedKernels2013_beta$med <- Med2013
    SimulatedKernels2013_beta$dist90 <- Dist90_2013
    SimulatedKernels2013_beta$iteration <- n
    
    
#join results into larger df
SimulatedKernels2013 <- bind_rows(SimulatedKernels2013, SimulatedKernels2013_beta)
    
setTxtProgressBar(pb, n)


}
close(pb)
EndTime <- Sys.time()
EndTime-StartTime
options(warn=0) #turn warnings back on


write.csv(SimulatedKernels2013, file="~/oceanography/script_output/KernelFits/Adjust_NoRecentRecruits/1000SimulatedKernels2013NoRecentRec.csv", row.names=F)

SimulatedKernels2014 <- as.data.frame(matrix(nrow=0, ncol=7), stringsAsFactors = FALSE)
colnames(SimulatedKernels2014) <- col

#don't print warnings for this loop, they are only for setting row names in a tibble which is depracated. But it works. 
options(warn=-1)

pb <- txtProgressBar(min = 0, max = 1000, style = 3)

StartTime <- Sys.time()

for(n in 1:1000){
        
    #SAMPLE THE SIMULATION DATA, then fit a kernel
    
    #make an empty dataframe with the same columns as original, this will be sampled particles
    SimulatedSampling2014 = Simulation2014[FALSE,]
    
    for(i in 1:nrow(NGenOffs2014)){ 
        
        destination_eval <- as.character(NGenOffs2014$site[i]) #pick out a destination site
        
        SimSampDestination <- Simulation2014NoRecentRec %>% 
                                filter(destination==destination_eval) %>%
                                sample_n(as.numeric(NGenOffs2014$n_offs_gen[i]),  replace=F) 
        
        SimulatedSampling2014 <- bind_rows(SimulatedSampling2014, SimSampDestination) #build into a sampled particle df
    }
    
    #assign a numeric ID for each row (which is a sampled particle)
    SimulatedSampling2014 <- SimulatedSampling2014 %>%
        mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))
    
    #nrow(SimulatedSampling2014)==sum(NGenOffs2014$n_offs_gen) #should be TRUE
    
    #now randomly assign parentage match status to some of these rows
    NumPar <- as.numeric(kernels %>%
        filter(Year=="2014") %>%
        select(NumParentageMatches))
    
    SimulatedSampling2014Par <- SimulatedSampling2014 %>%
        ungroup() %>%
        filter(source %in% NGenOffs2014$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
        sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
        mutate(Parentage=1) #%>% #assign these rows a positive match value
        #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
        #select(date, source, destination, Parentage) #drop columns that will result in repeats 
    
    #find the rows that haven't been assigned parentage
    SimulatedSampling2014_2 <- anti_join((SimulatedSampling2014 %>% ungroup()), SimulatedSampling2014Par, by=c("ParticleSampID")) 
    
    SimulatedSampling2014_2$Parentage <- 0 #add in the column for parentage
    
    #combine parentage and non-parentage dfs
    SimulatedSampling2014Par3 <- bind_rows(SimulatedSampling2014Par, SimulatedSampling2014_2) %>%
        mutate(YearSampled=as.character(YearSampled))
    
    
    #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSamp2014 <- PropSamp %>%  #add in all sampled sites as possible parent sites
                            filter(end_year =="2014" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSamp2014$destination <- PropSamp2014$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSampling2014Par3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in all surveyed sites
                TotalSimSamp2 <- left_join(PropSamp2014, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSampling2014Par3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum all the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummary2014 <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum all the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finally, join all of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for all sites
                PropSamp2014 <- PropSamp2014 %>%
                                rename(source="site")
                
                PropSamp2014$destination <- PropSamp2014$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSamp2014, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% NGenOffs2014$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummary2014 %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFull2014 <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFull2014) <- NULL

        #The full remade parentage matrix
        Assignments2014 <- SimDispMatFull2014
    
    #NOW FIT THE KERNEL


x <- list(Distances=Distances, Assignments=Assignments2014, Sampled_reefs=Sampled_reefs2014, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportions2014) #put inputs into a list because that's the bbmle format

Sim2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))

BestK2014 <- as.numeric(coef(Sim2014Fit)[1])
BestTheta2014 <- as.numeric(coef(Sim2014Fit)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
    
    #store the info in this df
    SimulatedKernels2014_beta <- as.data.frame(matrix(nrow=1, ncol=7), stringsAsFactors = FALSE)
    colnames(SimulatedKernels2014_beta) <- col
    
    SimulatedKernels2014_beta$year <- 2014
    SimulatedKernels2014_beta$k <- BestK2014
    SimulatedKernels2014_beta$theta <- BestTheta2014
    SimulatedKernels2014_beta$mdd <- MDD2014
    SimulatedKernels2014_beta$med <- Med2014
    SimulatedKernels2014_beta$dist90 <- Dist90_2014
    SimulatedKernels2014_beta$iteration <- n
    
    
#join results into larger df
SimulatedKernels2014 <- bind_rows(SimulatedKernels2014, SimulatedKernels2014_beta)
    
setTxtProgressBar(pb, n)


}
close(pb)
EndTime <- Sys.time()
EndTime-StartTime
options(warn=0) #turn warnings back on


write.csv(SimulatedKernels2014, file="~/oceanography/script_output/KernelFits/Adjust_NoRecentRecruits/1000SimulatedKernels2014NoRecentRec.csv", row.names=F)



  |======================================================================| 100%


Time difference of 1.623931 hours

  |=========================================                             |  59%

In [ ]:
#combine simulations
SimulationAllNoRecentRec <- bind_rows(Simulation2012, Simulation2013, Simulation2014)%>%
        filter(SimMonth %!in% c(4,5))#account for the fact that we didn't sample really new recruits

#make a DF of all of the sampled fish over 2012-2014
AllYearsRecruits <- AnnualRecsSamp %>% distinct(site, .keep_all = T)
sum(AllYearsRecruits$n_offs_gen_all_years) #should be 394 for 2012-2014

In [ ]:
#get together kernel fitting components for all years/seasonal

#assemble all of the components for annual kernel fitting
centroids <- read.csv("~/oceanography/empirical_data/site_centroids_simulation_kernels.csv", header=TRUE)

Area <- read.csv("~/oceanography/empirical_data/site_area_header_nonsurveyed_simulation_kernels.csv", header=TRUE) %>%
    arrange(site) %>%
    filter(site %!in% c("near_north_full1", "near_north_full2", "near_north_full3", "near_south_full1", "near_south_full2", "near_south_full3")) %>%
    mutate(kmsq=msq*10^-6) %>%
    select(kmsq)
Reef_sizes <- as.matrix(Area)

#give every site in the distance matrix of the simulation (even if we didn't sample there) an index number
AllSites <- centroids %>%
    select(site) %>%
    arrange(site)
nrow(AllSites) #should be 34x1
AllSites$index <- seq(from=1, to=34, by=1)

PropSampAll <- PropSamp %>%  #add in all sampled sites as possible parent sites
            filter(end_year =="2014" & PropSamp > 0) %>%
            rename(source="site") %>%
            select(-end_year, -PropSamp)

SampledSitesAll <- inner_join(PropSampAll %>% select(source), AllSites, by=c(source="site")) 

#check for correct number of rows
nrow(SampledSitesAll)==nrow(PropSampAll) #should be true
SampledSitesAllIndex <- t(as.matrix(SampledSitesAll$index))
ncol(SampledSitesAllIndex)

Sampled_reefsAll <- SampledSitesAllIndex

#proportion sampled matrix for kernel fitting

Adult_sample_proportionsAll <- PropSamp %>%  #add in all sampled sites as possible parent sites
                filter(end_year =="2014" & PropSamp > 0) 

Adult_sample_proportionsAll <- as.matrix(Adult_sample_proportionsAll$PropSamp)
nrow(Adult_sample_proportionsAll) 

#distance matrix using the centroids with combined Magbangon
### List of source locations
sites_source <- centroids

### List of destination locations
sites_dest <- centroids

dist_mat_m <- distm(sites_source[,c('lon','lat')], sites_source[,c('lon','lat')], fun=distVincentyEllipsoid)
Distances <- dist_mat_m*10^-3

Centroids <- centroids %>%
    select(-site)

#for the all years sites sampled in the simulation, it's a total of 16. That's lower than the all year used in kernel fitting because we didn't sample Sitio Tugas or Gabas until after 2014. We also combined the Magbangons, so 19-16=3 accounted for

In [ ]:
#All of the corresponding simulation years combined (2011-14)

col <- c("year", "k", "theta", "mdd", "med", "dist90", "iteration")
SimulatedKernelsAll <- as.data.frame(matrix(nrow=0, ncol=7), stringsAsFactors = FALSE)
colnames(SimulatedKernelsAll) <- col

#don't print warnings for this loop, they are only for setting row names in a tibble which is depracated. But it works. 
options(warn=-1)

pb <- txtProgressBar(min = 0, max = 1000, style = 3)#7 years in each interation

StartTime <- Sys.time()


for(n in 1:1000){
        
    #SAMPLE THE SIMULATION DATA, then fit a kernel
    
    SimulatedSamplingAll = SimulationAllNoRecentRec[FALSE,]
    
    for(i in 1:nrow(AllYearsRecruits)){ 
        
        destination_eval <- as.character(AllYearsRecruits$site[i]) #pick out a destination site
        
        SimSampDestination <- SimulationAllNoRecentRec %>%
                                    filter(destination==destination_eval) %>%
                                    sample_n(as.numeric(AllYearsRecruits$n_offs_gen_all_years[i]), replace=F) #sample particles that landed at that destination, number corresponding to the actual number sampled at that site in this year
        
        SimulatedSamplingAll <- bind_rows(SimulatedSamplingAll, SimSampDestination) #build into a sampled particle df
    }

    #assign a numeric ID for each row (which is a sampled particle)
    SimulatedSamplingAll <- SimulatedSamplingAll %>%
        mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))
    
    #nrow(SimulatedSamplingAll)==sum(NGenOffsAll$n_offs_gen) #should be TRUE
    
    #now randomly assign parentage match status to some of these rows
    NumPar <- as.numeric(kernels %>% #should be 37 for 2012-2014
        filter(Year %in% c("2012", "2013", "2014")) %>%
        select(NumParentageMatches) %>%
        summarise(NumParentageMatches=sum(NumParentageMatches)))
    
    SimulatedSamplingAllPar <- SimulatedSamplingAll %>%
        ungroup() %>%
        filter(source %in% NGenOffsAll$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
        sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
        mutate(Parentage=1) #%>% #assign these rows a positive match value
        #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
        #select(date, source, destination, Parentage) #drop columns that will result in repeats 
    
    #find the rows that haven't been assigned parentage
    SimulatedSamplingAll_2 <- anti_join((SimulatedSamplingAll %>% ungroup()), SimulatedSamplingAllPar, by=c("ParticleSampID")) 
    
    SimulatedSamplingAll_2$Parentage <- 0 #add in the column for parentage
    
    #combine parentage and non-parentage dfs
    SimulatedSamplingAllPar3 <- bind_rows(SimulatedSamplingAllPar, SimulatedSamplingAll_2) %>%
        mutate(YearSampled=as.character(YearSampled))
    
    
    #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSampAll <- PropSamp %>%  #add in all sampled sites as possible parent sites
                            filter(end_year =="2014" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSampAll$destination <- PropSampAll$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSamplingAllPar3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in all surveyed sites
                TotalSimSamp2 <- left_join(PropSampAll, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSamplingAllPar3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum all the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummaryAll <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum all the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finally, join all of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for all sites
                PropSampAll <- PropSampAll %>%
                                rename(source="site")
                
                PropSampAll$destination <- PropSampAll$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSampAll, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% NGenOffsAll$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummaryAll %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFullAll <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFullAll) <- NULL

        #The full remade parentage matrix
        AssignmentsAll <- SimDispMatFullAll
    
    #NOW FIT THE KERNEL


x <- list(Distances=Distances, Assignments=AssignmentsAll, Sampled_reefs=Sampled_reefsAll, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportionsAll) #put inputs into a list because that's the bbmle format

SimAllFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))

BestKAll <- as.numeric(coef(SimAllFit)[1])
BestThetaAll <- as.numeric(coef(SimAllFit)[2])
MDDAll <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKAll, theta=BestThetaAll, , method = "pcubature")$integral)
k_eval <- BestKAll
theta_eval <- BestThetaAll
MedAll  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_All <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
    
    #store the info in this df
    SimulatedKernelsAll_beta <- as.data.frame(matrix(nrow=1, ncol=7), stringsAsFactors = FALSE)
    colnames(SimulatedKernelsAll_beta) <- col
    
    SimulatedKernelsAll_beta$year <- NA
    SimulatedKernelsAll_beta$k <- BestKAll
    SimulatedKernelsAll_beta$theta <- BestThetaAll
    SimulatedKernelsAll_beta$med <- MedAll
    SimulatedKernelsAll_beta$mdd <- MDDAll
    SimulatedKernelsAll_beta$dist90 <- Dist90_All
    SimulatedKernelsAll_beta$iteration <- n
    
    
#join results into larger df
SimulatedKernelsAll <- bind_rows(SimulatedKernelsAll, SimulatedKernelsAll_beta)
    
setTxtProgressBar(pb, n)


}
close(pb)
EndTime <- Sys.time()
EndTime-StartTime
options(warn=0) #turn warnings back on


write.csv(SimulatedKernelsAll, file="~/oceanography/script_output/KernelFits/Adjust_NoRecentRecruits/1000SimulatedKernelsAllNoRecentRec.csv", row.names=F)

__Seasonal kernel fits__

In [ ]:
SimulationSWMNoRecentRec <- SimulationAllNoRecentRec %>%
    filter(SimMonsoon=="SWM")
SimulationNEMNoRecentRec <- SimulationAllNoRecentRec %>%
    filter(SimMonsoon=="NEM")

In [ ]:
#SWM of the corresponding simulation years combined (2011-SWM)

col <- c("season", "k", "theta", "mdd", "med", "dist90", "iteration")
SimulatedKernelsSWM <- as.data.frame(matrix(nrow=0, ncol=7), stringsAsFactors = FALSE)
colnames(SimulatedKernelsSWM) <- col

#don't print warnings for this loop, they are only for setting row names in a tibble which is depracated. But it works. 
options(warn=-1)

pb <- txtProgressBar(min = 0, max = 1000, style = 3)

StartTime <- Sys.time()


for(n in 1:1000){
        
    #SAMPLE THE SIMULATION DATA, then fit a kernel
    
    SimulatedSamplingSWM = SimulationSWMNoRecentRec[FALSE,]
    
    
    for(i in 1:nrow(AllYearsRecruits)){ 
        
        destination_eval <- as.character(AllYearsRecruits$site[i]) #pick out a destination site
        
        SimSampDestination <- SimulationSWMNoRecentRec %>%
                                    filter(destination==destination_eval) %>%
                                    sample_n(as.numeric(AllYearsRecruits$n_offs_gen_all_years[i]), replace=F) #sample particles that landed at that destination, number corresponding to the actual number sampled at that site in this year
        
        SimulatedSamplingSWM <- bind_rows(SimulatedSamplingSWM, SimSampDestination) #build into a sampled particle df
    }

    #assign a numeric ID for each row (which is a sampled particle)
    SimulatedSamplingSWM <- SimulatedSamplingSWM %>%
        mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))
    
    #nrow(SimulatedSamplingSWM)==sum(NGenOffsSWM$n_offs_gen) #should be TRUE
    
   #now randomly assign parentage match status to some of these rows
    NumPar <- as.numeric(kernels %>% #should be 37 for 2012-2014
        filter(Year %in% c("2012", "2013", "2014")) %>%
        select(NumParentageMatches) %>%
        summarise(NumParentageMatches=sum(NumParentageMatches)))
    
    SimulatedSamplingSWMPar <- SimulatedSamplingSWM %>%
        ungroup() %>%
        filter(source %in% AllYearsRecruits$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
        sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
        mutate(Parentage=1) #%>% #assign these rows a positive match value
        #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
        #select(date, source, destination, Parentage) #drop columns that will result in repeats 
    
    #find the rows that haven't been assigned parentage
    SimulatedSamplingSWM_2 <- anti_join((SimulatedSamplingSWM %>% ungroup()), SimulatedSamplingSWMPar, by=c("ParticleSampID")) 
    
    SimulatedSamplingSWM_2$Parentage <- 0 #add in the column for parentage
    
    #combine parentage and non-parentage dfs
    SimulatedSamplingSWMPar3 <- bind_rows(SimulatedSamplingSWMPar, SimulatedSamplingSWM_2) %>%
        mutate(YearSampled=as.character(YearSampled))
    
    
    #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSampSWM <- PropSamp %>%  #add in SWM sampled sites as possible parent sites
                            filter(end_year =="2014" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSampSWM$destination <- PropSampSWM$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSamplingSWMPar3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in SWM surveyed sites
                TotalSimSamp2 <- left_join(PropSampSWM, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSamplingSWMPar3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum SWM the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummarySWM <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum SWM the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finSWMy, join SWM of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for SWM sites
                PropSampSWM <- PropSampSWM %>%
                                rename(source="site")
                
                PropSampSWM$destination <- PropSampSWM$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSampSWM, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% AllYearsRecruits$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummarySWM %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFullSWM <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFullSWM) <- NULL

        #The full remade parentage matrix
        AssignmentsSWM <- SimDispMatFullSWM
    
    #NOW FIT THE KERNEL


x <- list(Distances=Distances, Assignments=AssignmentsSWM, Sampled_reefs=Sampled_reefsAll, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportionsAll) #put inputs into a list because that's the bbmle format

SimSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))

BestKSWM <- as.numeric(coef(SimSWMFit)[1])
BestThetaSWM <- as.numeric(coef(SimSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
    
    #store the info in this df
    SimulatedKernelsSWM_beta <- as.data.frame(matrix(nrow=1, ncol=7), stringsAsFactors = FALSE)
    colnames(SimulatedKernelsSWM_beta) <- col
    
    SimulatedKernelsSWM_beta$season <- NA
    SimulatedKernelsSWM_beta$k <- BestKSWM
    SimulatedKernelsSWM_beta$theta <- BestThetaSWM
    SimulatedKernelsSWM_beta$med <- MedSWM
    SimulatedKernelsSWM_beta$mdd <- MDDSWM
    SimulatedKernelsSWM_beta$dist90 <- Dist90_SWM
    SimulatedKernelsSWM_beta$iteration <- n

    
#join results into larger df
SimulatedKernelsSWM <- bind_rows(SimulatedKernelsSWM, SimulatedKernelsSWM_beta) %>%
    mutate(season="SWM")
    
setTxtProgressBar(pb, n)


}
close(pb)
EndTime <- Sys.time()
EndTime-StartTime
options(warn=0) #turn warnings back on


write.csv(SimulatedKernelsSWM, file="~/oceanography/script_output/KernelFits/Adjust_NoRecentRecruits/1000SimulatedKernelsSWMNoRecentRec.csv", row.names=F)

In [ ]:
#NEM of the corresponding simulation years combined (2011-NEM)

col <- c("season", "k", "theta", "mdd", "med", "dist90", "iteration")
SimulatedKernelsNEM <- as.data.frame(matrix(nrow=0, ncol=7), stringsAsFactors = FALSE)
colnames(SimulatedKernelsNEM) <- col

#don't print warnings for this loop, they are only for setting row names in a tibble which is depracated. But it works. 
options(warn=-1)

pb <- txtProgressBar(min = 0, max = 1000, style = 3)

StartTime <- Sys.time()


for(n in 1:1000){
        
    #SAMPLE THE SIMULATION DATA, then fit a kernel
    
    SimulatedSamplingNEM = SimulationNEMNoRecentRec[FALSE,]
    
    
    for(i in 1:nrow(AllYearsRecruits)){ 
        
        destination_eval <- as.character(AllYearsRecruits$site[i]) #pick out a destination site
        
        SimSampDestination <- SimulationNEMNoRecentRec %>%
                                    filter(destination==destination_eval) %>%
                                    sample_n(as.numeric(AllYearsRecruits$n_offs_gen_all_years[i]), replace=F) #sample particles that landed at that destination, number corresponding to the actual number sampled at that site in this year
        
        SimulatedSamplingNEM <- bind_rows(SimulatedSamplingNEM, SimSampDestination) #build into a sampled particle df
    }

    #assign a numeric ID for each row (which is a sampled particle)
    SimulatedSamplingNEM <- SimulatedSamplingNEM %>%
        mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))
    
    #nrow(SimulatedSamplingNEM)==sum(NGenOffsNEM$n_offs_gen) #should be TRUE
    
   #now randomly assign parentage match status to some of these rows
    NumPar <- as.numeric(kernels %>% #should be 37 for 2012-2014
        filter(Year %in% c("2012", "2013", "2014")) %>%
        select(NumParentageMatches) %>%
        summarise(NumParentageMatches=sum(NumParentageMatches)))
    
    SimulatedSamplingNEMPar <- SimulatedSamplingNEM %>%
        ungroup() %>%
        filter(source %in% AllYearsRecruits$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
        sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
        mutate(Parentage=1) #%>% #assign these rows a positive match value
        #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
        #select(date, source, destination, Parentage) #drop columns that will result in repeats 
    
    #find the rows that haven't been assigned parentage
    SimulatedSamplingNEM_2 <- anti_join((SimulatedSamplingNEM %>% ungroup()), SimulatedSamplingNEMPar, by=c("ParticleSampID")) 
    
    SimulatedSamplingNEM_2$Parentage <- 0 #add in the column for parentage
    
    #combine parentage and non-parentage dfs
    SimulatedSamplingNEMPar3 <- bind_rows(SimulatedSamplingNEMPar, SimulatedSamplingNEM_2) %>%
        mutate(YearSampled=as.character(YearSampled))
    
    
    #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSampNEM <- PropSamp %>%  #add in NEM sampled sites as possible parent sites
                            filter(end_year =="2014" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSampNEM$destination <- PropSampNEM$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSamplingNEMPar3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in NEM surveyed sites
                TotalSimSamp2 <- left_join(PropSampNEM, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSamplingNEMPar3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum NEM the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummaryNEM <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum NEM the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finNEMy, join NEM of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for NEM sites
                PropSampNEM <- PropSampNEM %>%
                                rename(source="site")
                
                PropSampNEM$destination <- PropSampNEM$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSampNEM, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% AllYearsRecruits$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummaryNEM %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFullNEM <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFullNEM) <- NULL

        #The full remade parentage matrix
        AssignmentsNEM <- SimDispMatFullNEM
    
    #NOW FIT THE KERNEL


x <- list(Distances=Distances, Assignments=AssignmentsNEM, Sampled_reefs=Sampled_reefsAll, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportionsAll) #put inputs into a list because that's the bbmle format

SimNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))

BestKNEM <- as.numeric(coef(SimNEMFit)[1])
BestThetaNEM <- as.numeric(coef(SimNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
    
    #store the info in this df
    SimulatedKernelsNEM_beta <- as.data.frame(matrix(nrow=1, ncol=7), stringsAsFactors = FALSE)
    colnames(SimulatedKernelsNEM_beta) <- col
    
    SimulatedKernelsNEM_beta$season <- NA
    SimulatedKernelsNEM_beta$k <- BestKNEM
    SimulatedKernelsNEM_beta$theta <- BestThetaNEM
    SimulatedKernelsNEM_beta$med <- MedNEM
    SimulatedKernelsNEM_beta$mdd <- MDDNEM
    SimulatedKernelsNEM_beta$dist90 <- Dist90_NEM
    SimulatedKernelsNEM_beta$iteration <- n

    
#join results into larger df
SimulatedKernelsNEM <- bind_rows(SimulatedKernelsNEM, SimulatedKernelsNEM_beta) %>%
    mutate(season="NEM")
    
setTxtProgressBar(pb, n)


}
close(pb)
EndTime <- Sys.time()
EndTime-StartTime
options(warn=0) #turn warnings back on


write.csv(SimulatedKernelsNEM, file="~/oceanography/script_output/KernelFits/Adjust_NoRecentRecruits/1000SimulatedKernelsNEMNoRecentRec.csv", row.names=F)

In [30]:
head(SimulatedKernels2012)
head(SimulatedKernels2013)
head(SimulatedKernels2014)
head(SimulatedKernelsAll)
head(SimulatedKernelsNEM)
head(SimulatedKernelsSWM)

year,k,theta,mdd,med,dist90,iteration
2012,-2.4366998,1.4242511,7.834620,6.11,17.02,1
2012,-2.4205348,1.2909762,8.375805,6.35,18.49,2
2012,-0.9350943,0.5525553,10.366102,5.24,25.93,3
2012,-1.0174343,0.5686806,10.169503,5.24,25.37,4
2012,2.3653021,0.2628124,46.142707,10.77,111.03,5
2012,2.3253602,0.2613187,50.958911,11.79,122.43,6


year,k,theta,mdd,med,dist90,iteration
2013,-0.4116770,0.5338819,6.974513,3.44,17.50,1
2013,-1.6267518,0.8022077,7.419344,4.67,17.70,2
2013,-1.5718681,0.7560483,7.968735,4.87,19.18,3
2013,-0.4118165,0.5148600,8.034851,3.86,20.22,4
2013,-1.4005995,0.7356399,7.146876,4.30,17.27,5
2013,0.3033083,0.4158950,10.710904,4.30,27.20,6


year,k,theta,mdd,med,dist90,iteration
2014,-3.302688,8.0000000,13.08272,12.80,23.91,1
2014,-3.326996,8.0000000,13.40464,13.12,24.50,2
2014,-2.893352,1.9168909,10.36991,8.69,21.51,3
2014,-3.329090,8.0000000,13.43272,13.15,24.55,4
2014,-1.218194,0.5498559,14.00419,7.05,35.04,5
2014,-1.290937,0.5672648,13.48421,6.94,33.64,6


year,k,theta,mdd,med,dist90,iteration
NA,-2.30492,0.9733976,10.42441,7.15,24.11,1


season,k,theta,mdd,med,dist90,iteration
NEM,-2.132757,0.8685141,10.5742,6.91,24.91,1
NEM,-1.439111,0.6254059,11.4177,6.26,28.18,2


season,k,theta,mdd,med,dist90,iteration
SWM,-2.566107,1.446359,8.813001,6.90,19.09,1
SWM,-2.847139,2.068605,9.598297,8.17,19.69,2
SWM,-2.693211,1.810927,8.716944,7.22,18.23,3
